<a href="https://colab.research.google.com/github/JimHuynh25/CS5588-group2/blob/main/CS5588_Week3_HandsOn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 5588 — Week 3 Hands-On  
## Building a Multimodal RAG Product Prototype (PDF + Images)

**Goal (today):** Build a *working product prototype* that answers user questions from real documents (PDFs + images) with **evidence citations**.

**What you’ll leave with:**
- A project-ready multimodal RAG pipeline (ingestion → indexing → retrieval → grounded answer)
- A short **Product Brief** inside the notebook (persona, problem, value, success metrics)
- A small **demo loop** you can show to stakeholders (prompt → answer + citations)

> This hands-on is application-first: prioritize a realistic use case and a clean demo.


## 0) Product Brief (Fill in — REQUIRED for Week 3)
- **Team / Name:**  
- **Project name (working title):**  

### 0.1 Target user persona
- Who will use this? (role, context, pain point)

### 0.2 Problem statement (1–2 sentences)
- What decision/task does your product support?

### 0.3 Value proposition (1 sentence)
- What improves (speed, accuracy, trust, cost, risk)?

### 0.4 Success metrics (pick 2–3)
- e.g., time-to-answer, citation coverage, % “not enough evidence” when missing, user satisfaction (1–5), precision@5


## 1) Setup (Colab)
Run installs, then imports.


In [1]:
# === Setup & Imports (Colab-friendly) ===
import os, re, glob, json, math
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple, Optional

import numpy as np
import pandas as pd

# ---- Core deps ----
# PyMuPDF for PDF text extraction
!pip -q install pymupdf pillow pandas numpy scikit-learn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

import fitz  # PyMuPDF
from PIL import Image

# ---- OCR deps ----
!pip -q install pytesseract
!sudo apt-get -qq update
!sudo apt-get -qq install -y tesseract-ocr

import pytesseract

# ---- Retrieval deps ----
!pip -q install faiss-cpu rank-bm25
import faiss
from rank_bm25 import BM25Okapi

# ---- Dense + rerank (optional) ----
# Some environments may have version conflicts. We try to install, but fall back gracefully if needed.
USE_ST = True
USE_RERANK = True

try:
    from sentence_transformers import SentenceTransformer, CrossEncoder
except Exception as e:
    USE_ST = False
    USE_RERANK = False
    print("⚠️ sentence-transformers not available in this runtime. Falling back to TF-IDF for 'dense' retrieval.")
    print("   Error:", e)

# Optional captioning (bonus)
USE_CAPTIONING = False
try:
    from transformers import pipeline
    USE_CAPTIONING = True
except Exception:
    USE_CAPTIONING = False


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


### 1.1 System dependencies (Colab/Linux)
If OCR fails, run this cell.


In [22]:
# (Handled in Setup & Imports above)
print('System dependencies installed in Section 1.')

System dependencies installed in Section 1.


### 1.2 Imports


> **Note:** Dependencies are installed and imported above. If you restart the runtime, re-run Sections 1–2.

## 2) Choose a project dataset (realistic, stakeholder-facing)
Create this structure (you can start small today):

```
project_data_mm/
  docs/
    doc1.pdf
    doc2.pdf
  figures/
    fig1.png
    fig2.jpg
  notes.txt (optional)
```

**Recommended today:** 2 PDFs + 3–5 images that matter to your use case.


In [23]:
import os, glob, zipfile, shutil

DATA_DIR = "project_data_mm"
DOC_DIR = os.path.join(DATA_DIR, "docs")
FIG_DIR = os.path.join(DATA_DIR, "figures")
TMP_DIR = os.path.join(DATA_DIR, "_tmp_unzip")

os.makedirs(DOC_DIR, exist_ok=True)
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(TMP_DIR, exist_ok=True)

# ✅ Colab uploads are here
zip_files = sorted(glob.glob("/content/*.zip"))
print("Using ZIPs:", zip_files)

IMG_EXTS = {".png", ".jpg", ".jpeg", ".webp", ".bmp", ".tif", ".tiff"}
PDF_EXTS = {".pdf"}

# unzip
for z in zip_files:
    with zipfile.ZipFile(z, "r") as zip_ref:
        zip_ref.extractall(TMP_DIR)

# move into docs/figures
moved_pdf = moved_img = 0
for root, _, files in os.walk(TMP_DIR):
    for fn in files:
        src = os.path.join(root, fn)
        ext = os.path.splitext(fn.lower())[1]

        if ext in PDF_EXTS:
            shutil.move(src, os.path.join(DOC_DIR, fn))
            moved_pdf += 1
        elif ext in IMG_EXTS:
            shutil.move(src, os.path.join(FIG_DIR, fn))
            moved_img += 1

print(f"Moved PDFs: {moved_pdf}, Moved images: {moved_img}")

# verify
pdfs = sorted(glob.glob(os.path.join(DOC_DIR, "*.pdf")))
imgs = sorted(glob.glob(os.path.join(FIG_DIR, "*.*")))
print("PDFs:", len(pdfs), pdfs[:5])
print("Images:", len(imgs), imgs[:5])


Using ZIPs: ['/content/pictrues.zip', '/content/withour 2000.zip']
Moved PDFs: 14, Moved images: 10
PDFs: 7 ['project_data_mm/docs/2025ccfsrumkcpolice.pdf', 'project_data_mm/docs/2026-spring-shuttle-schedule.pdf', 'project_data_mm/docs/Student Permits - Parking Options - Parking _ University of Missouri-Kansas City.pdf', 'project_data_mm/docs/umkc-health-sciences-campus-map.pdf', 'project_data_mm/docs/umkc-student-handbook.pdf']
Images: 5 ['project_data_mm/figures/1.png', 'project_data_mm/figures/2.png', 'project_data_mm/figures/3.png', 'project_data_mm/figures/4.png', 'project_data_mm/figures/5.png']


## 3) Define 3 stakeholder questions (application-oriented)
- **Q1/Q2:** require both text + figure/table evidence  
- **Q3:** ambiguous/missing evidence → system should say **Not enough evidence in the retrieved context.**

Also add:
- Must-cite evidence (page or figure)
- Success criteria (what a good answer must include)


In [24]:
QUERIES = [
    {
        "id": "Q1",
        "question": (
            "Which shuttle bus station is closest to Plaster Hall "
            "according to the campus map and shuttle route information?"
        ),
        "must_cite": ["[map]", "[route]"],
        "success_criteria": [
            "Mentions a specific shuttle stop",
            "Uses campus map evidence",
            "Cites shuttle route or map figure"
        ],
        "keywords": ["Plaster Hall", "shuttle", "map", "route"]
    },
    {
        "id": "Q2",
        "question": (
            "Which parking lot is closest to Haag Hall, and how much does "
            "a student pay for a semester parking permit for that lot?"
        ),
        "must_cite": ["[parking map]", "[parking table]"],
        "success_criteria": [
            "Identifies the nearest parking lot",
            "States a numeric semester permit price",
            "Uses both text and table evidence"
        ],
        "keywords": ["Haag Hall", "parking", "permit", "price"]
    },
    {
        "id": "Q3",
        "question": (
            "Will UMKC’s student enrollment decrease in 2026 compared to previous years?"
        ),
        "must_cite": [],
        "success_criteria": [
            "Not enough evidence in the retrieved context."
        ],
        "keywords": ["enrollment", "2026", "trend"]
    },
]


## 4) Ingest PDFs (per-page text)


In [25]:
from dataclasses import dataclass
from typing import List
import os, re
import fitz  # PyMuPDF

@dataclass
class TextChunk:
    chunk_id: str
    doc_id: str
    page_num: int
    text: str
    source: str  # file path for citations

def extract_pdf_pages(pdf_path: str) -> List[TextChunk]:
    doc_id = os.path.basename(pdf_path)
    out: List[TextChunk] = []

    try:
        with fitz.open(pdf_path) as doc:
            for i in range(len(doc)):
                page = doc.load_page(i)
                text = page.get_text("text") or ""
                text = re.sub(r"\s+", " ", text).strip()

                if text:
                    out.append(
                        TextChunk(
                            chunk_id=f"{doc_id}::p{i+1}",
                            doc_id=doc_id,
                            page_num=i+1,
                            text=text,
                            source=pdf_path
                        )
                    )
    except Exception as e:
        print(f"[WARN] Failed to read {pdf_path}: {e}")

    return out

page_chunks = []
for p in pdfs:
    page_chunks.extend(extract_pdf_pages(p))

print("Total PDF page chunks:", len(page_chunks))
if page_chunks:
    print("Sample:", page_chunks[0].chunk_id, page_chunks[0].text[:250])


Total PDF page chunks: 256
Sample: 2025ccfsrumkcpolice.pdf::p1 1 UNIVERSITY OF MISSOURI – KANSAS CITY JEANNE CLERY ACT REPORT Annual Campus Security and Fire Safety Report for 2024 Reported September 2025 In accordance with the Jeanne Clery Disclosure of Campus Security Policy and Campus Crime Statistics Act of 


## 5) Ingest images (OCR first, optional captioning)


In [6]:
@dataclass
class EvidenceItem:
    evid_id: str
    source: str
    image_path: str
    ocr_text: str
    caption_text: str
    evidence_text: str

def run_ocr(image_path: str) -> str:
    img = Image.open(image_path).convert("RGB")
    text = pytesseract.image_to_string(img)
    return re.sub(r"\s+", " ", text).strip()

evidence_items = []
for ip in imgs:
    base = os.path.basename(ip)
    evid_id = os.path.splitext(base)[0]
    ocr = run_ocr(ip)
    evidence_items.append(EvidenceItem(evid_id, base, ip, ocr, "", ocr))

print("Evidence items:", len(evidence_items))
if evidence_items:
    print("Sample OCR:", evidence_items[0].source, evidence_items[0].ocr_text[:200])


Evidence items: 5
Sample OCR: 1.png Undergraduate Research and Creative Scholarship This year we celebrate 50 years of being at the forefront of Undergraduate Research and Creative Scholarship, which offers undergraduate students opport


### 5.1 Optional captioning (bonus)


In [7]:
### 5.1 Optional captioning (bonus)

USE_CAPTIONING = False  # set True to enable

if USE_CAPTIONING:
    from transformers import pipeline
    from PIL import Image
    import re

    # Use GPU if available (Colab)
    captioner = pipeline(
        "image-to-text",
        model="Salesforce/blip-image-captioning-base",
        device=0
    )

    for ei in evidence_items:
        try:
            img = Image.open(ei.image_path).convert("RGB")

            cap = captioner(
                img,
                max_new_tokens=40
            )[0]["generated_text"]

            cap = re.sub(r"\s+", " ", cap).strip()

            # Store caption separately
            ei.caption_text = cap

            # Merge OCR + caption safely
            if ei.ocr_text:
                ei.evidence_text = (ei.ocr_text + "\n" + cap).strip()
            else:
                ei.evidence_text = cap

        except Exception as e:
            print(f"[WARN] Captioning failed for {ei.image_path}: {e}")

    print("Captioning complete.")

else:
    print("Captioning skipped.")


Captioning skipped.


## 6) Chunking (page-based vs fixed-size)


In [8]:
@dataclass
class SubChunk:
    chunk_id: str
    doc_id: str
    page_num: int
    text: str

def fixed_size_chunk(text: str, words_per_chunk: int = 250, overlap: int = 40) -> List[str]:
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = min(len(words), start + words_per_chunk)
        chunks.append(" ".join(words[start:end]))
        if end == len(words):
            break
        start = max(0, end - overlap)
    return chunks

sub_chunks = []
for pc in page_chunks:
    for j, t in enumerate(fixed_size_chunk(pc.text, 250, 40)):
        sub_chunks.append(SubChunk(f"{pc.doc_id}::p{pc.page_num}::c{j+1}", pc.doc_id, pc.page_num, t))

print("Page chunks:", len(page_chunks))
print("Fixed-size chunks:", len(sub_chunks))


Page chunks: 256
Fixed-size chunks: 464


## 7) Indexing & retrieval (dense + sparse + rerank)


In [9]:
def tokenize(text: str) -> List[str]:
    return [t.lower() for t in re.findall(r"[a-zA-Z0-9]+", text)]

# --- Embeddings (dense retrieval) ---
# If SentenceTransformers is available, we use it. Otherwise, we fall back to TF-IDF vectors.
if USE_ST:
    embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    def embed_texts(texts: List[str], batch_size: int = 32) -> np.ndarray:
        return embedder.encode(
            texts, batch_size=batch_size, show_progress_bar=True,
            convert_to_numpy=True, normalize_embeddings=True
        )
else:
    # TF-IDF fallback (acts as a "dense-ish" baseline)
    tfidf_vec = TfidfVectorizer(max_features=50000, ngram_range=(1,2))
    _tfidf_fitted = False

    def embed_texts(texts: List[str], batch_size: int = 32) -> np.ndarray:
        global _tfidf_fitted
        X = tfidf_vec.fit_transform(texts) if not _tfidf_fitted else tfidf_vec.transform(texts)
        _tfidf_fitted = True
        X = normalize(X)
        return X.toarray().astype(np.float32)

def build_faiss_ip(vectors: np.ndarray):
    dim = vectors.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(vectors.astype(np.float32))
    return index

TEXT_CORPUS_A = page_chunks
TEXT_CORPUS_B = sub_chunks

texts_A = [c.text for c in TEXT_CORPUS_A]
vecs_A = embed_texts(texts_A) if texts_A else np.zeros((0,384), dtype=np.float32)
faiss_A = build_faiss_ip(vecs_A) if len(texts_A)>0 else None
bm25_A = BM25Okapi([tokenize(t) for t in texts_A]) if len(texts_A)>0 else None

texts_B = [c.text for c in TEXT_CORPUS_B]
vecs_B = embed_texts(texts_B) if texts_B else np.zeros((0,384), dtype=np.float32)
faiss_B = build_faiss_ip(vecs_B) if len(texts_B)>0 else None
bm25_B = BM25Okapi([tokenize(t) for t in texts_B]) if len(texts_B)>0 else None

evid_texts = [e.evidence_text for e in evidence_items]
evid_vecs = embed_texts(evid_texts) if evid_texts else np.zeros((0,384), dtype=np.float32)
faiss_E = build_faiss_ip(evid_vecs) if len(evid_texts)>0 else None

print("Indexes ready.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Indexes ready.


In [10]:
def dense_search(query: str, index, corpus, top_k: int = 5):
    if index is None or len(corpus)==0:
        return []
    qv = embed_texts([query])
    scores, idxs = index.search(qv.astype(np.float32), top_k)
    out = []
    for s, i in zip(scores[0], idxs[0]):
        if int(i) >= 0:
            out.append((float(s), corpus[int(i)]))
    return out

def sparse_search(query: str, bm25, corpus, top_k: int = 5):
    if bm25 is None or len(corpus)==0:
        return []
    scores = bm25.get_scores(tokenize(query))
    top = np.argsort(scores)[::-1][:top_k]
    return [(float(scores[i]), corpus[int(i)]) for i in top]

def hybrid_fuse(dense_res, sparse_res, alpha: float = 0.5, top_k: int = 5):
    def k(item): return getattr(item, "chunk_id", getattr(item, "evid_id", str(item)))
    dense_rank = {k(it): r for r, (_, it) in enumerate(dense_res, start=1)}
    sparse_rank = {k(it): r for r, (_, it) in enumerate(sparse_res, start=1)}
    keys = set(dense_rank) | set(sparse_rank)
    fused = []
    for key in keys:
        dr = dense_rank.get(key, len(dense_res)+1)
        sr = sparse_rank.get(key, len(sparse_res)+1)
        score = alpha*(1.0/dr) + (1-alpha)*(1.0/sr)
        obj = next((it for _, it in dense_res if k(it)==key), None) or next((it for _, it in sparse_res if k(it)==key), None)
        fused.append((score, obj))
    fused.sort(key=lambda x: x[0], reverse=True)
    return fused[:top_k]

# --- Reranker (optional) ---
reranker = None
if USE_ST and USE_RERANK:
    try:
        reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    except Exception as e:
        reranker = None
        USE_RERANK = False
        print("⚠️ Reranker unavailable, continuing without reranking. Error:", e)


def rerank(query: str, items, get_text, top_k=5):
    if reranker is None:
        return list(items)[:top_k]

    if not items:
        return []
    scores = reranker.predict([(query, get_text(it)) for it in items])
    ranked = sorted(zip(scores, items), key=lambda x: x[0], reverse=True)
    return [it for _, it in ranked[:top_k]]

def retrieve_text(query: str, chunking: str = "page", method: str = "hybrid", top_k: int = 5, alpha: float = 0.5, use_rerank: bool = True):
    if chunking == "page":
        corpus, index, bm25 = TEXT_CORPUS_A, faiss_A, bm25_A
    else:
        corpus, index, bm25 = TEXT_CORPUS_B, faiss_B, bm25_B

    if method == "dense":
        res = dense_search(query, index, corpus, top_k=max(10, top_k))
        items = [it for _, it in res]
    elif method == "sparse":
        res = sparse_search(query, bm25, corpus, top_k=max(10, top_k))
        items = [it for _, it in res]
    else:
        d = dense_search(query, index, corpus, top_k=max(10, top_k))
        s = sparse_search(query, bm25, corpus, top_k=max(10, top_k))
        res = hybrid_fuse(d, s, alpha=alpha, top_k=max(10, top_k))
        items = [it for _, it in res]

    if use_rerank:
        return rerank(query, items, lambda it: it.text, top_k=top_k)
    return items[:top_k]

def retrieve_evidence(query: str, top_k: int = 3):
    res = dense_search(query, faiss_E, evidence_items, top_k=top_k)
    return [it for _, it in res]


Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


## 8) Evidence pack + citations (product output)


In [11]:
def cite_text(it): return f"[{it.doc_id} p{it.page_num}]"
def cite_fig(ei): return f"[{os.path.splitext(ei.source)[0]}]"

def build_evidence_pack(question: str, chunking="page", method="hybrid", top_k_text=4, top_k_fig=2):
    txt = retrieve_text(question, chunking=chunking, method=method, top_k=top_k_text, use_rerank=True)
    figs = retrieve_evidence(question, top_k=top_k_fig)
    pack = []
    for it in txt:
        pack.append({"type":"text", "cite": cite_text(it), "content": it.text[:800]})
    for ei in figs:
        pack.append({"type":"figure", "cite": cite_fig(ei), "content": (ei.evidence_text or "")[:800], "path": ei.image_path})
    return pack

ep = build_evidence_pack(QUERIES[0]["question"])
for e in ep:
    print(e["cite"], e["type"], e["content"][:120])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[umkc-student-handbook.pdf p16] text Return to Table of Contents P a g e 16 Graduation Requirements To stay on trach to meet all graduation requirements, stu
[umkc-student-handbook.pdf p46] text Return to Table of Contents P a g e 46 The UMKC O365 email account is the used for University business and all official 
[umkc-student-handbook.pdf p15] text Return to Table of Contents P a g e 15 Academic Resources UMKC provides a wide variety of resources to help students suc
[umkc-student-handbook.pdf p11] text Return to Table of Contents P a g e 11 Learning Environment UMKC strives to maintain the best possible learning environm
[5] figure Reimagining Higher Education Le S200M+ Annual financial assistance We're opening the doors of opportunity to more studen
[4] figure Top Rankings Among the best in the Midwest The Princeton Review describes UMKC as "academically outstanding and well wor


## 9) Grounded response (LLM/VLM) — connect Gemini/HF if available


In [17]:
from typing import Optional
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"  # fast for Colab demos

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,        # prevents max_length=20 conflicts
    do_sample=False,
    temperature=0.0,
    pad_token_id=tokenizer.pad_token_id
)

def generate_answer(prompt: str, image_paths: Optional[list] = None) -> str:
    out = generator(prompt, max_new_tokens=160)  # smaller = faster
    text = out[0]["generated_text"]
    return text.replace(prompt, "").strip()

print("CUDA available:", torch.cuda.is_available())


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Passing `generation_config` together with generation-related arguments=({'max_new_tokens', 'temperature', 'pad_token_id', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


## 10) Demo loop (stakeholder-facing)


In [21]:
import re

def rag_prompt(question: str, evidence_pack: list) -> str:
    evidence_lines = []
    for e in (evidence_pack or []):
        cite = e.get("cite", "[unknown]")
        content = e.get("content", "")
        content = re.sub(r"\s+", " ", str(content)).strip()
        if content:
            evidence_lines.append(f"{cite} {content}")

    evidence_block = "\n\n".join(evidence_lines)

    return f"""You are a grounded assistant. Use ONLY the evidence below.
Every key claim must cite evidence like [doc p#] or [fig1].
If the evidence is insufficient, respond exactly:
Not enough evidence in the retrieved context.

Evidence:
{evidence_block}

Question:
{question}

Answer (with citations):
"""


In [19]:
def demo_one(question: str, chunking="page", method="hybrid"):
    ep = build_evidence_pack(question, chunking=chunking, method=method) or []
    prompt = rag_prompt(question, ep)

    image_paths = []
    for e in ep:
        if e.get("type") == "figure" and e.get("path"):
            image_paths.append(e["path"])

    ans = generate_answer(prompt, image_paths=image_paths)
    return ep, ans
for q in QUERIES:
    ep, ans = demo_one(q["question"], chunking="page", method="hybrid")

    print("\n=== ", q["id"], " ===")
    print("Q:", q["question"])
    print("Top evidence citations:", [e.get("cite") for e in ep])
    print("Answer:", (ans or "")[:500])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


KeyboardInterrupt: 

## 11) Week 3 acceptance tests (CS 5588)
Fill in after running your demo:
- Does the evidence pack include the must-cite items for Q1/Q2?
- Does Q3 properly refuse with “Not enough evidence…”?
- Is the output understandable to your target user?


In [ ]:
ACCEPTANCE_CHECKLIST = [
    {
        "qid": "Q1",
        "must_cite_expected": "[map] + [route]",
        "pass_fail": "TODO",
        "notes": "PASS if answer names a stop + cites map/route."
    },
    {
        "qid": "Q2",
        "must_cite_expected": "[parking map] + [parking table] (with numeric price)",
        "pass_fail": "TODO",
        "notes": "PASS if nearest lot + semester price + table citation."
    },
    {
        "qid": "Q3",
        "must_cite_expected": "(none) — should refuse",
        "pass_fail": "TODO",
        "notes": "PASS only if exact refusal string is returned."
    },
]
ACCEPTANCE_CHECKLIST


## 11.5 Team work items (project enhancement)

Use this hands-on to **advance your semester project**. Each team member should “own” at least one deliverable below.

**Product Lead (Applicability)**
- Update your project **persona + workflow** so the multimodal RAG module is a *core feature*, not an add-on.
- Write 3 stakeholder tasks that map to your product’s real decision points (2 require text+figure evidence, 1 must refuse).

**Systems Lead (Integration)**
- Replace the toy dataset with your **project-domain PDFs + figures**.
- Add **metadata fields** that matter to your domain (e.g., policy date, version, department, study cohort, device model).
- Implement a clean **`retrieve()` API** your final demo can reuse.

**Evaluation & Risk Lead (Shipping readiness)**
- Build a tiny evaluation table: *Task × Method × P@5 × R@10 × Faithfulness*.
- Add one real failure scenario + mitigation UX (warnings, “show evidence” first, or human-in-the-loop flag).
- Draft the “If we shipped this” plan: data refresh, monitoring, and governance rule.

**Bonus (Optional)**
- Add a minimal UI (Gradio/Streamlit) that shows: question → evidence pack → answer with citations.


## 12) Week 3 deliverables (CS 5588)
- Product Brief completed (persona, problem, value, success metrics)
- Demo run for Q1–Q3 with citations (screenshots encouraged)
- 1 failure case + mitigation plan (risk + fix)
- Repo link submitted in the survey
